## Часть 2



- Я хочу проанализировать все возможные характеристики для KNN-графа и выбрать, какая из них работает лучше. Это минимальная и макимальная степени, число точек сочленения, число треугольников и число компонент связности.

In [39]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import importlib
import functions
importlib.reload(functions)

<module 'functions' from '/Users/berdov/dm_pr/dm/src/functions.py'>

In [40]:
from functions import max_degree
from functions import min_degree
from functions import count_articulation_points
from functions import count_triangles
from functions import count_components

- Теперь я подключу функции генерации распределений и многократного проведения эксперимента


In [41]:
from functions import (
    build_knn_graph,
    sample_normal,
    sample_t,
    monte_carlo_characteristic
)

- Теперь все готово к анализу, какая же характеристика лучше

In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt


n = 500
k = 10
n_sim = 300
alpha = 0.05


characteristics = {
    'Макс. степень': max_degree,
    'Мин. степень': min_degree,
    'Компоненты': count_components,
    'Сочленения': count_articulation_points,
    'Треугольники': count_triangles
}

summary = []


for name, func in characteristics.items():
    print(f"Анализ: {name}")

    T_H0 = monte_carlo_characteristic(sample_normal, lambda X: build_knn_graph(X, k), func, 1, n=n, n_sim=n_sim)
    T_H1 = monte_carlo_characteristic(sample_t, lambda X: build_knn_graph(X, k), func, 3, n=n, n_sim=n_sim)

    threshold = np.percentile(T_H0, 100 * (1 - alpha))
    power = np.mean(T_H1 > threshold)
    auc = roc_auc_score([0] * len(T_H0) + [1] * len(T_H1), np.concatenate([T_H0, T_H1]))

    summary.append({
        'Характеристика': name,
        'AUC ROC': round(auc, 3),
        'Power (H1)': round(power, 3),
        'Порог (95% H0)': round(threshold, 2)
    })


df_summary = pd.DataFrame(summary)
print(df_summary)

Анализ: Макс. степень
Анализ: Мин. степень
Анализ: Компоненты
Анализ: Сочленения
Анализ: Треугольники
  Характеристика  AUC ROC  Power (H1)  Порог (95% H0)
0  Макс. степень    0.594       0.020           19.00
1   Мин. степень    0.500       0.000           10.00
2     Компоненты    0.507       0.000            2.00
3     Сочленения    0.512       0.020            3.00
4   Треугольники    0.683       0.127         7547.05


# Вывод для KNN-графа

- отвратительно отработали абсолютно все арактеристики, буду все переделывать. Теперь буду строить DIST-граф и считать у него хроматическое число, кликовое число, размер максимального независимого множества, доминирующее число, размер минимального кликового покрытия

In [43]:
from functions import chromatic_number
from functions import clique_number
from functions import max_independent_set_size
from functions import domination_number
from functions import clique_cover_number

- повторим анализ аналогичным образом

In [44]:
from functions import (
    build_dist_graph,
    sample_normal,
    sample_t,
    monte_carlo_characteristic
)


from networkx.algorithms import approximation as approx

In [45]:
n = 500
d = 0.3
n_sim = 200
alpha = 0.05

characteristics = {
    'Хроматическое число': chromatic_number,
    'Кликовое число': clique_number,
    'Макс. независимое множество': max_independent_set_size,
    'Доминирование': domination_number,
    'Кликовое покрытие': clique_cover_number
}

summary = []

for name, func in characteristics.items():
    print(f"Анализ: {name}")

    T_H0 = monte_carlo_characteristic(sample_normal, lambda X: build_dist_graph(X, d), func, 1, n=n, n_sim=n_sim)
    T_H1 = monte_carlo_characteristic(sample_t, lambda X: build_dist_graph(X, d), func, 3, n=n, n_sim=n_sim)

    threshold = np.percentile(T_H0, 100 * (1 - alpha))
    power = np.mean(T_H1 > threshold)
    auc = roc_auc_score([0] * len(T_H0) + [1] * len(T_H1), np.concatenate([T_H0, T_H1]))

    summary.append({
        'Характеристика': name,
        'AUC ROC': round(auc, 3),
        'Power (H1)': round(power, 3),
        'Порог (95% H0)': round(threshold, 2)
    })

df_summary = pd.DataFrame(summary)
print(df_summary)

Анализ: Хроматическое число
Анализ: Кликовое число
Анализ: Макс. независимое множество
Анализ: Доминирование
Анализ: Кликовое покрытие


AttributeError: 'function' object has no attribute 'clique_removal'